# Customer Data Cleaning Notebook

This notebook connects to a PostgreSQL database, loads customer churn data, performs essential data cleaning, and exports the cleaned data.

## 1. Setup & Connection

Import necessary libraries and establish database connection using SQLAlchemy.

In [3]:
# Import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Load environment variables from .env file
load_dotenv('../.env')

# Database connection parameters from .env file
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'telco_churn')

# Validate that required credentials are set
if not DB_USER or not DB_PASSWORD:
    raise ValueError('DB_USER and DB_PASSWORD must be set in .env file')

# Create SQLAlchemy connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create engine
engine = create_engine(connection_string)
print(f"Connection engine created successfully!")
print(f"  Database: {DB_NAME}")
print(f"  Host: {DB_HOST}:{DB_PORT}")

Connection engine created successfully!
  Database: postgres
  Host: localhost:5432


## 2. Load Data

Query the entire `customers` table and load it into a Pandas DataFrame.

In [6]:
# Load the customers table into a DataFrame
query = "SELECT * FROM public.customers;"

try:
    df = pd.read_sql(query, engine)
    print(f"Data loaded successfully! Shape: {df.shape}")
except Exception as e:
    print(f"Error loading data: {e}")

Data loaded successfully! Shape: (7043, 21)


## 3. Exploratory Data Analysis (EDA)

### 3.1 Display First Rows

In [7]:
# Display the first few rows
print("First 5 rows of the dataset:")
df.head()

First 5 rows of the dataset:


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,7590-VHVEG,Female,False,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,False,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,False,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,False,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,False,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [8]:
# Display the last few rows
print("Last 5 rows of the dataset:")
df.tail()

Last 5 rows of the dataset:


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
7038,6840-RESVB,Male,False,True,True,24,True,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,True,Mailed check,84.80,1990.50,False
7039,2234-XADUH,Female,False,True,True,72,True,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,True,Credit card (automatic),103.20,7362.90,False
7040,4801-JZAZL,Female,False,True,True,11,False,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,True,Electronic check,29.60,346.45,False
7041,8361-LTMKD,Male,True,True,False,4,True,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Mailed check,74.40,306.60,True
7042,3186-AJIEK,Male,False,False,False,66,True,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,True,Bank transfer (automatic),105.65,6844.50,False


### 3.2 Data Types and Structure

In [9]:
# Display DataFrame info (data types, non-null counts)
print("DataFrame Info:")
print("=" * 50)
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   gender             7043 non-null   object 
 2   senior_citizen     7043 non-null   bool   
 3   partner            7043 non-null   bool   
 4   dependents         7043 non-null   bool   
 5   tenure             7043 non-null   int64  
 6   phone_service      7043 non-null   bool   
 7   multiple_lines     7043 non-null   object 
 8   internet_service   7043 non-null   object 
 9   online_security    7043 non-null   object 
 10  online_backup      7043 non-null   object 
 11  device_protection  7043 non-null   object 
 12  tech_support       7043 non-null   object 
 13  streaming_tv       7043 non-null   object 
 14  streaming_movies   7043 non-null   object 
 15  contract           7043 non-null   object 
 16  paperles

In [10]:
# Display basic statistics
print("Descriptive Statistics:")
df.describe(include='all')

Descriptive Statistics:


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
count,7043,7043,7043,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043,7043,7043.000000,7032.000000,7043
unique,7043,2,2,2,2,NaN,2,3,3,3,...,3,3,3,3,3,2,4,NaN,NaN,2
top,7590-VHVEG,Male,False,False,False,NaN,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,NaN,NaN,False
freq,1,3555,5901,3641,4933,NaN,6361,3390,3096,3498,...,3095,3473,2810,2785,3875,4171,2365,NaN,NaN,5174
mean,NaN,NaN,NaN,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,2283.300441,NaN
std,NaN,NaN,NaN,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,2266.771362,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,18.800000,NaN
25%,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,401.450000,NaN
50%,NaN,NaN,NaN,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,1397.475000,NaN
75%,NaN,NaN,NaN,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,3794.737500,NaN


In [11]:
# Show data types
print("Data Types:")
print("=" * 50)
df.dtypes

Data Types:


customer_id           object
gender                object
senior_citizen          bool
partner                 bool
dependents              bool
tenure                 int64
phone_service           bool
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_tv          object
streaming_movies      object
contract              object
paperless_billing       bool
payment_method        object
monthly_charges      float64
total_charges        float64
churn                   bool
dtype: object

## 4. Data Cleaning

### 4.1 Check for Missing/Null Values

In [12]:
# Check for null values in each column
print("Null Values per Column:")
print("=" * 50)
null_counts = df.isnull().sum()
null_percentages = (df.isnull().sum() / len(df)) * 100

null_summary = pd.DataFrame({
    'Null Count': null_counts,
    'Null Percentage (%)': null_percentages.round(2)
})
null_summary[null_summary['Null Count'] > 0]

Null Values per Column:


,Null Count,Null Percentage (%)
total_charges,11,0.16


In [13]:
# Display total null values
total_nulls = df.isnull().sum().sum()
print(f"Total null values in dataset: {total_nulls}")

Total null values in dataset: 11


In [17]:
# Handle missing values
# Strategy: 
# - For numeric columns (monthly_charges, total_charges): fill with median
# - For boolean columns: fill with mode (most frequent value)
# - For text columns: fill with 'Unknown' or mode

print("Handling missing values...")
print(f"Shape before: {df.shape}")

# Numeric columns - fill with median
numeric_cols = ['monthly_charges', 'total_charges', 'tenure']
for col in numeric_cols:
    if col in df.columns and df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"  - Filled '{col}' nulls with median: {median_val}")

# Boolean columns - fill with mode
boolean_cols = ['senior_citizen', 'partner', 'dependents', 'phone_service', 
                'paperless_billing', 'churn']
for col in boolean_cols:
    if col in df.columns and df[col].isnull().any():
        mode_val = df[col].mode()[0] if not df[col].mode().empty else False
        df[col].fillna(mode_val, inplace=True)
        print(f"  - Filled '{col}' nulls with mode: {mode_val}")

# Text columns - fill with 'Unknown'
text_cols = ['gender', 'multiple_lines', 'internet_service', 'online_security',
             'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
             'streaming_movies', 'contract', 'payment_method']
for col in text_cols:
    if col in df.columns and df[col].isnull().any():
        df[col].fillna('Unknown', inplace=True)
        print(f"  - Filled '{col}' nulls with 'Unknown'")

# For customer_id - drop rows with null (primary key cannot be null)
if 'customer_id' in df.columns and df['customer_id'].isnull().any():
    df.dropna(subset=['customer_id'], inplace=True)
    print("  - Dropped rows with null customer_id")

print(f"Shape after: {df.shape}")

Handling missing values...
Shape before: (7043, 21)
Shape after: (7043, 21)


In [18]:
# Verify no null values remain
remaining_nulls = df.isnull().sum().sum()
print(f"Remaining null values after cleaning: {remaining_nulls}")

Remaining null values after cleaning: 0


### 4.2 Check for Duplicates

In [19]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Check for duplicate customer_ids (should be unique as it's primary key)
duplicate_ids = df['customer_id'].duplicated().sum()
print(f"Number of duplicate customer_ids: {duplicate_ids}")

Number of duplicate rows: 0
Number of duplicate customer_ids: 0


In [20]:
# Display duplicate rows if any exist
if duplicate_count > 0:
    print("Duplicate rows found:")
    duplicates = df[df.duplicated(keep=False)]
    display(duplicates)

In [21]:
# Remove duplicate rows
print(f"Shape before removing duplicates: {df.shape}")

# Remove exact duplicate rows (keep first occurrence)
df.drop_duplicates(inplace=True)

# Remove duplicate customer_ids (keep first occurrence)
df.drop_duplicates(subset=['customer_id'], keep='first', inplace=True)

print(f"Shape after removing duplicates: {df.shape}")

Shape before removing duplicates: (7043, 21)
Shape after removing duplicates: (7043, 21)


### 4.3 Verify Object Columns

In [37]:
# 4.3.1 - Check unique values in object (categorical) columns
print("Unique values in object (categorical) columns:")
print("=" * 60)

object_columns = df.select_dtypes(include='object').columns

for col in object_columns:
    unique_vals = df[col].unique()
    print(f"\t{col} ({len(unique_vals)} unique values):")
    print(f"  {unique_vals}")

Unique values in object (categorical) columns:
	customer_id (7043 unique values):
  ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
	gender (2 unique values):
  ['Female' 'Male']
	multiple_lines (3 unique values):
  ['No phone service' 'No' 'Yes']
	internet_service (3 unique values):
  ['DSL' 'Fiber optic' 'No']
	online_security (3 unique values):
  ['No' 'Yes' 'No internet service']
	online_backup (3 unique values):
  ['Yes' 'No' 'No internet service']
	device_protection (3 unique values):
  ['No' 'Yes' 'No internet service']
	tech_support (3 unique values):
  ['No' 'Yes' 'No internet service']
	streaming_tv (3 unique values):
  ['No' 'Yes' 'No internet service']
	streaming_movies (3 unique values):
  ['No' 'Yes' 'No internet service']
	contract (3 unique values):
  ['Month-to-month' 'One year' 'Two year']
	payment_method (4 unique values):
  ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


In [39]:
# 4.3.4 - Convert Yes/No/No service columns to 0/1
service_columns = ['multiple_lines', 'online_security', 'online_backup', 
                   'device_protection', 'tech_support', 'streaming_tv', 
                   'streaming_movies']

for col in service_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 1 if x == 'Yes' else 0)

print("✓ Service columns converted to 0/1")

✓ Service columns converted to 0/1


### 4.3 Type Conversion

Ensure column types match the database schema logic.

In [40]:
# Convert boolean columns to 0/1 
boolean_columns = ['senior_citizen', 'partner', 'dependents', 'phone_service',
                   'paperless_billing', 'churn','multiple_lines', 'online_security', 'online_backup', 
                   'device_protection', 'tech_support', 'streaming_tv', 
                   'streaming_movies']

for col in boolean_columns:
    df[col] = df[col].astype(int)

print("✓ Boolean columns converted to 0/1")

✓ Boolean columns converted to 0/1


In [41]:
# Verify data types after conversion
print("\nData types after conversion:")
print("=" * 50)
df.dtypes


Data types after conversion:


customer_id           object
gender                object
senior_citizen         int64
partner                int64
dependents             int64
tenure                 int64
phone_service          int64
multiple_lines         int64
internet_service      object
online_security        int64
online_backup          int64
device_protection      int64
tech_support           int64
streaming_tv           int64
streaming_movies       int64
contract              object
paperless_billing      int64
payment_method        object
monthly_charges      float64
total_charges        float64
churn                  int64
dtype: object

### 4.4 Final Data Validation

In [42]:
# Final validation summary
print("=" * 60)
print("DATA CLEANING SUMMARY")
print("=" * 60)
print(f"Total records: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"Remaining null values: {df.isnull().sum().sum()}")
print(f"Remaining duplicates: {df.duplicated().sum()}")
print("=" * 60)

DATA CLEANING SUMMARY
Total records: 7043
Total columns: 21
Remaining null values: 0
Remaining duplicates: 0


In [43]:
# Display final info
print("Final DataFrame Info:")
df.info()

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   gender             7043 non-null   object 
 2   senior_citizen     7043 non-null   int64  
 3   partner            7043 non-null   int64  
 4   dependents         7043 non-null   int64  
 5   tenure             7043 non-null   int64  
 6   phone_service      7043 non-null   int64  
 7   multiple_lines     7043 non-null   int64  
 8   internet_service   7043 non-null   object 
 9   online_security    7043 non-null   int64  
 10  online_backup      7043 non-null   int64  
 11  device_protection  7043 non-null   int64  
 12  tech_support       7043 non-null   int64  
 13  streaming_tv       7043 non-null   int64  
 14  streaming_movies   7043 non-null   int64  
 15  contract           7043 non-null   object 
 16  pa

## 5. Export Cleaned Data

In [44]:
# Save the cleaned data to CSV
output_path = '../data/cleaned_customers.csv'

df.to_csv(output_path, index=False)
print(f"✓ Cleaned data saved to: {output_path}")
print(f"  - Total records: {len(df)}")
print(f"  - Total columns: {len(df.columns)}")

✓ Cleaned data saved to: ../data/cleaned_customers.csv
  - Total records: 7043
  - Total columns: 21


In [45]:
# Verify the saved file
df_verify = pd.read_csv(output_path)
print(f"Verification - Loaded {len(df_verify)} rows from saved file.")
print("\nFirst 3 rows of saved file:")
df_verify.head(3)

Verification - Loaded 7043 rows from saved file.

First 3 rows of saved file:


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,7590-VHVEG,Female,0,1,0,1,0,0,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,0,0,34,1,0,DSL,1,...,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,0,0,2,1,0,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1


## 6. Close Database Connection

In [ ]:
# Close the database connection
engine.dispose()
print("Database connection closed.")

---
## Summary

This notebook performed the following data cleaning operations:

1. **Connected** to PostgreSQL database using SQLAlchemy (credentials loaded from `.env` file)
2. **Loaded** the `customers` table into a Pandas DataFrame
3. **Explored** the data structure (head, tail, info, describe)
4. **Cleaned** the data:
   - Identified and handled missing values
   - Removed duplicate rows
   - Converted data types to match the database schema
5. **Exported** the cleaned data to `cleaned_customers.csv`